In [1]:
# библиотеки для работы с dataset
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

# опции для вывода данных в нужном количестве и формате
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

# системная библиотека
import sys

# библиотеки для ML
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder

# библиотеки для парсинга сайта
import requests, json

# библиотека для перевода слов
import goslate
gs = goslate.Goslate()

# библиотеки для визуализации
import altair as alt
# отключение ограничения в 5000 строк для alt
alt.data_transformers.disable_max_rows()
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# библиотеки для статистического анализа
from scipy.stats import ttest_ind
from scipy.stats import chi2_contingency
from scipy.stats import chi2

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
def description_column(x):
    
    """Функция используется для описания 
     столбца текстового типа """
    
    print('Количество уникальных значений в столбце:',x.nunique())
    print('Перечень уникальных значений:', x.unique())
    print('Длина столбца:',len(x))
    print('Количество значений "NaN":', x.isna().sum())
    print("Количество значений, встретившихся в столбце более 10 раз:", (x.value_counts()>10).sum())

def description_column_count(x):
    
    """Функция используется для описания 
     столбца числового типа """
    
    print('Длина столбца:',len(x))
    print('Количество значений "NaN":', x.isna().sum())
    print('Минимальное значение:', x.min())
    print('Максимальное значение:',x.max())  
    
def outliers(x): 
    
    """Функция используется для 
    определения границ выбросов"""
    
    median = x.median()
    IQR = x.quantile(0.75)-x.quantile(0.25)
    q25 = x.quantile(0.25)
    q75 = x.quantile(0.75)
    print('25-й квартиль: {},'.format(q25), '75-й квартиль: {},'.format(q75), 'IQR: {},'.format(IQR),
     "границы выбросов: [{x_min},{x_max}].".format(x_min = q25-1.5*IQR, x_max = q75+1.5*IQR))

def delete_outliers(x): 
    
    """Функция используется для 
    очистки данных от выбросов"""
    
    median = x.median()
    IQR = x.quantile(0.75)-x.quantile(0.25)
    q25 = x.quantile(0.25)
    q75 = x.quantile(0.75)
    x_min =q25-1.5*IQR
    x_max =q75+1.5*IQR
    return x_min, x_max
    
def boxplot(x1,y1,d,t,tx,ty):
    
    """Функция используется для 
    вывода boxplot по нескольким признакам"""
    
    red_square = dict(markerfacecolor='k', marker='s')
    plt.figure(figsize=(18, 5))
    plt.style.use('seaborn')
    sns.boxplot(x=x1, y=y1, data=d, linewidth=1, flierprops=red_square)
    plt.title(t, fontsize=15, color='black')
    plt.xlabel(tx, fontsize=15, color='black')
    plt.ylabel(ty, fontsize=15, color='black')
    plt.tick_params(axis='both', labelsize=15)
    plt.grid(True)
    

def one_boxplot(x, title, x_title):
    
    """Функция используется для 
    вывода boxplot по одному признаку"""
    
    red_square = dict(markerfacecolor='k', marker='s')
    plt.figure(figsize=(18, 5))
    plt.style.use('seaborn')
    plt.tick_params(axis='both', labelsize=15)
    ax=sns.boxplot(x, linewidth=1, flierprops=red_square)
    ax.set_title(title, fontsize=15, color='black')
    ax.set_xlabel(x_title, fontsize=15, color='black')

def correlation(x,y):
    
    """Функция используется для 
    расчета коэффициента корреляции""" 
        
    display(pd.DataFrame(all_data[[x,y]].corr()))

def get_stat_dif(column):
    cols = all_data.loc[:, column].value_counts().index[:10]
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(all_data.loc[all_data.loc[:, column] == comb[0], 'score'], 
                        all_data.loc[all_data.loc[:, column] == comb[1], 'score']).pvalue \
            <= 0.05/len(combinations_all): # Учли поправку Бонферони
            print('Найдены статистически значимые различия для колонки', column)
            break

def cm_to_inch(value):
    return value/2.54

def bar_pivot_table(x,z,y,c,t,xl,yl):
    
    """Функция используется для 
    вывода диаграммы по одному признаку"""
    
    plt.style.use('seaborn')
    plt.rcParams["figure.figsize"] = (20,7)
    all_data.pivot_table(values = [x, z], index = [y], aggfunc = c).plot(kind = 'area', title = t)
    plt.xlabel(xl, fontsize=15, color='black')
    plt.ylabel(yl, fontsize=15, color='black')
    plt.title(t, fontsize=15, color='black')
    plt.tick_params(axis='both', labelsize=15)
    plt.legend(loc='lower right', facecolor='black', fontsize=15)
    plt.legend()
    plt.grid(False)
    plt.show()

def area_two_pivot_table(x,z,y,c,t,xl,yl):
    
    """Функция используется для 
    вывода диаграммы по одному признаку"""
    
    plt.style.use('seaborn')
    plt.rcParams["figure.figsize"] = (20,7)
    all_data.pivot_table(values = [x], index = [y], columns=[z], aggfunc = c).plot(kind = 'area',stacked=True,title = t)
    plt.xlabel(xl, fontsize=15, color='black')
    plt.ylabel(yl, fontsize=15, color='black')
    plt.legend()
    plt.grid(False)
    plt.show()
    
def share(x, x_name, y_name, title):
    
    """Функция используется для 
    вывода диаграммы по долям"""
    
    new=pd.DataFrame(x.value_counts(normalize = True)*100).reset_index(level=0).head(10)
    new.columns=([x_name, y_name])
    plt.style.use('seaborn')
    plt.figure(figsize=(18, 5))
    plt.tick_params(axis='both', labelsize=15)
    ax=sns.barplot(x=x_name, y=y_name, data=new)   
    ax.set_ylabel(y_name, fontsize=15, color='black')
    ax.set_xlabel(x_name, fontsize=15, color='black')
    ax.set_title(title, fontsize=15, color='black')
    for counter, value in enumerate(new[y_name]):
        ax.text(counter, value, round(value), ha='left', size=16)


def share_count(x, x_name, y_name, title):
    
    """Функция используется для 
    вывода диаграммы по долям"""
    
    new=pd.DataFrame(x.value_counts()).reset_index(level=0).head(10).sort_values(by=['index'])
    new.columns=([x_name, y_name])
    plt.style.use('seaborn')
    plt.figure(figsize=(18, 5))
    plt.tick_params(axis='both', labelsize=15)
    ax=sns.barplot(x=x_name, y=y_name, data=new)
    ax.set_ylabel(y_name, fontsize=15, color='black')
    ax.set_xlabel(x_name, fontsize=15, color='black')
    ax.set_title(title, fontsize=15, color='black')
    for counter, value in enumerate(new[y_name]):
        ax.text(counter, value, round(value), ha='center', size=16)

def bar_groupby(x,y,z,t,xl,yl):
    
    """Функция используется для 
    вывода диаграммы по нескольким признакам"""
    
    plt.style.use('seaborn')
    plt.rcParams["figure.figsize"] = (20,7)
    all_data.groupby(by=[x,y])[z].value_counts(normalize=True).plot(kind='bar', title=t)
    plt.xlabel(xl, fontsize=15, color='black')
    plt.ylabel(yl, fontsize=15, color='black')
    plt.tick_params(axis='both', labelsize=15)
    plt.grid(True)
    plt.show()

def hist_(x):
    plt.style.use('seaborn')
    plt.tick_params(axis='both', labelsize=15)
    plt.hist(x)
    
def correlation_vis(x_name, y_name, title):
    
    """Функция используется для 
    вывода графика распределения"""
    
    plt.style.use('seaborn')
    ax = sns.jointplot(x=x_name, y=y_name, data=df_corr, kind='scatter')

def altair_bar(data, x_name, y_name, legend_name, title_name):
    
    bar = alt.Chart(data,title=title_name).mark_bar(size=100).encode(
    x=alt.X(x_name, axis=alt.Axis(title=x_name, labelFontSize=13, titleFontSize=13, labelAngle=360)),
    y=alt.Y(y_name, axis=alt.Axis(title=y_name, labelFontSize=13, titleFontSize=13)),
    text=y_name,
    color=alt.value("SteelBlue"),
    size=alt.value(13)
    )

    text = bar.mark_text(
    align='center',
    baseline='middle',
    angle=0,
    dy=-5).encode(
    text=alt.Text(y_name, format="s"),
    color=alt.value("black"))
    
    return (bar + text).properties(height=250, width=840)

def altair_bar_share(data, x_name, y_name, title_name, sort):
    
    new=pd.DataFrame(data.value_counts(normalize = True)).reset_index(level=0).head(10).sort_values(by=sort, 
                                                                                                    ascending=False)
    new.columns=([x_name, y_name])
    
    bar = alt.Chart(new, title=title_name).mark_bar(orient='horizontal').encode(
    y=alt.Y(x_name, axis=alt.Axis(title=x_name, labelFontSize=13, titleFontSize=13)),
    x=alt.X(y_name, axis=alt.Axis(title=y_name, format='%', labelFontSize=13, titleFontSize=13)),
    text=y_name,
    color=alt.value("SteelBlue"),
    size=alt.value(13)
    )

    text = bar.mark_text(
    align='center',
    baseline='middle',
    angle=0,
    dx=40).encode(
    text=alt.Text(y_name, format="p"),
    color=alt.value("black"))
    
    return (bar + text).properties(height=250, width=820)

def hist(data, x1, x2, ylabel, xlabel, title):
    bins = np.arange(0, x1, x2)
    plt.style.use('seaborn')
    _, ax = plt.subplots(figsize=(18, 5))
    plt.tick_params(axis='both', labelsize=15)
    ax.hist(data, bins = bins)
    ax.set_ylabel(ylabel, fontsize=15, color='black')
    ax.set_xlabel(xlabel, fontsize=15, color='black')
    ax.set_title(title, fontsize=15, color='black')

In [3]:
# фиксация версий пакетов, для воспроизводимсти экспериментов
!pip freeze > requirements.txt

In [4]:
# фиксация random_seed для воспроивзодимости экспериментов
RANDOM_SEED = 42

In [5]:
VAL_SIZE = 0.20

In [6]:
train = pd.read_csv('train.csv', sep=';', decimal=',')
test = pd.read_csv('test.csv', sep=',', decimal=',')
sample_submission = pd.read_csv('sample_submission.csv', sep=',', decimal=',')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (17,19,23,26,29,33,73) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Список полей:

* date – Дата наблюдения, в формате ДД.ММ.ГГГГ
* region – Код региона в кодировке ФНС России
* okato  – ОКАТО региона РФ
* oktmo – ОКТМО региона РФ
* pasta  – макароны (средняя взвешенная цена), руб/кг
* legumes – бобовые,          руб/кг
* bread  – хлеб , руб/кг
* flour   – мука, руб/кг
* rice    – рис, руб/кг
* groats – другие крупы, руб/кг
* potatoes – картофель, руб/кг
* сucumbers_tomatoes – огурцы и помидоры, руб/кг
* vegetables – прочие овощи, руб/кг
* roots  – столовые корнеплоды, руб/кг
* сabbage – капуста, руб/кг
* fruit – фрукты, руб/кг
* sugar – сахар, руб/кг
* candy – конфеты, руб/кг
* biscuits – печенье, руб/кг
* mutton – баранина, руб/кг
* beef – говядина, руб/кг
* chicken – мясо птицы, руб/кг
* pork – свинина, руб/кг
* fish – рыба свеж, руб/кг
* herring – сельдь, руб/кг
* curd – творог, руб/кг
* sour_creame – сметана, руб/кг
* butter – масло сливочное, руб/кг
* milk_kefir – молоко кефир, руб/л
* cheese – сыр, руб/кг
* egg – яйца, руб/шт
* margarine – маргарин и другие жиры, руб/кг
* oil – масло раститительное, руб/л
* salt – соль, руб/кг
* spice –специи, руб/шт
* tea – чай, руб/шт
* CPI_1 – стоимость потребительской корзины для трудоспособных граждан, руб
* CPI_2 – стоимость потребительской корзины для пенсионеров, руб
* CPI_3 – стоимость потребительской корзины для детей до 18 лет, руб
* Pasta_value – макароны (объем проданной продукции), кг
* legumes_value – бобовые (объем проданной продукции), кг
* bread_value – хлеб (объем проданной продукции), кг
* flour_value – мука (объем проданной продукции), кг
* rice_value – рис (объем проданной продукции), кг
* groats_value – другие крупы (объем проданной продукции), кг
* potatoes_value – картофель (объем проданной продукции), кг
* сucumbers_tomatoes_value – огурцы и помидоры (объем проданной продукции), кг
* vegetables_value – прочие овощи (объем проданной продукции), кг
* roots_value – столовые корнеплоды (объем проданной продукции), кг
* сabbage_value – капуста (объем проданной продукции), кг
* fruit_value – фрукты (объем проданной продукции), кг
* sugar_value – сахар (объем проданной продукции), кг
* candy_value – конфеты (объем проданной продукции), кг
* biscuits_value         – печенье (объем проданной продукции), кг
* mutton_value – баранина (объем проданной продукции), кг
* beef_value – говядина (объем проданной продукции), кг
* chicken_value         – мясо птицы(объем проданной продукции), кг
* pork_value – свинина (объем проданной продукции), кг
* fish_value – рыба свеж (объем проданной продукции), кг
* herring_value – сельдь (объем проданной продукции), кг
* curd_value – творог (объем проданной продукции), кг
* sour_creame_value – сметана (объем проданной продукции), кг
* butter_value – масло сливочное (объем проданной продукции), кг
* milk_kefir_value –молоко кефир (объем проданной продукции), л
* cheese_value – сыр (объем проданной продукции), кг
* egg_value – яйца (объем проданной продукции), шт
* margarine_value – маргарин и другие жиры (объем проданной продукции), кг
* oil_value – масло раститительное (объем проданной продукции), л
* salt_value – соль (объем проданной продукции)        , кг
* spice_value – специи (объем проданной продукции), шт
* tea_value – чай (объем проданной продукции), шт
* ai92 – бензин марки АИ-92, руб/литр
* ai95 – бензин марки АИ-95, руб/литр
* ai98 – бензин марки АИ-98, руб/литр
* dt – дизельное топливо, руб/литр
* ai92_value – бензин марки АИ-92 (объем проданной продукции), литр
* ai95_value – бензин марки АИ-95 (объем проданной продукции), литр
* ai98_value – бензин марки АИ-98 (объем проданной продукции), литр
* dt_value – дизельное топливо (объем проданной продукции), литр

In [7]:
train['sample']='train'
test['sample']='test'
data = train.append(test, sort=False).reset_index(drop=True)

In [8]:
data.head()

,region,oktmo,okato,date,pasta,legumes,bread,flour,rice,groats,potatoes,сucumbers_tomatoes,vegetables,roots,сabbage,fruit,sugar,candy,biscuits,mutton,beef,chicken,pork,fish,herring,curd,sour_creame,butter,milk_kefir,cheese,egg,margarine,oil,salt,spice,tea,cpi_1,cpi_2,cpi_3,pasta_value,legumes_value,bread_value,flour_value,rice_value,groats_value,potatoes_value,сucumbers_tomatoes_value,vegetables_value,roots_value,сabbage_value,fruit_value,sugar_value,candy_value,biscuits_value,mutton_value,beef_value,chicken_value,pork_value,fish_value,herring_value,curd_value,sour_creame_value,butter_value,milk_kefir_value,cheese_value,egg_value,margarine_value,oil_value,salt_value,spice_value,tea_value,ai92,ai95,ai98,dt,ai92_value,ai95_value,ai98_value,dt_value,sample
0,72,71000000000,71000000000,31.03.2021,86.36,108.63,75.10,34.06,81.14,65.37,43.83,142.83,33.99,65.91,27.23,97.2,45.94,"290,59",182.61,0,396.89,171.12,251.04,"422,55",211.48,271.26,190.73,473.74,52.21,"542,24",7.54,137.39,113.56,18.47,37.06,82.58,"5 807,71","4 940,15","5 786,12","37 667,88","7 354,73","112 205,44","28 079,96","10 642,98","23 199,52","48 157,50","53 229,66","30 340,27","17 718,94","19 411,61","115 300,10","58 939,81","18 288,85","24 781,65",0,"2 559,60","72 052,29","15 490,87","1 340,24","4 150,95","14 712,82","17 137,00","6 445,98","182 550,44","14 711,66","571 499,52","806,44","27 602,00","9 965,01","40 875,05","44 193,60",43.40,48.18,54.85,50.18,"738 049,00","438 485,91","3 182,57","66 982,78",train
1,75,76000000000,76000000000,31.03.2021,83.48,123.53,73.87,42.34,65.98,50.66,50.86,189.45,37.05,66.51,33.59,134.2,50.61,"226,55",178.15,0,277.68,182.51,269.61,"518,62",241.57,332.37,235.74,514.16,63.21,"517,17",8.25,115.14,118.13,17.9,32.71,83.57,"6 312,35","5 352,06","6 495,01","22 049,69","2 385,52","36 299,15","13 586,97","5 843,72","13 730,06","8 161,46","10 229,90","9 835,79","6 033,72","6 486,47","24 966,00","17 986,08","9 495,51","14 761,89",0,"1 374,15","20 237,29","3 474,57","230,05","2 376,92","3 228,54","3 534,29","2 219,30","76 369,06","6 276,98","174 505,78","1 478,29","11 126,36","5 147,21","21 705,41","26 263,81",49.09,50.63,53.71,53.58,"350 912,44","144 884,38","7 341,17","36 931,85",train
2,74,75000000000,75000000000,31.03.2021,75.59,103.61,81.43,33.27,77.01,70.49,39.78,141.62,34.11,67.62,23.93,94.2,46.51,"288,54",137.63,"297,06",345.96,169.86,244.00,"298,66",179.46,277.84,184.06,421.86,55.50,"497,22",7.22,139.95,107.72,16.52,35.88,75.55,"5 541,37","4 703,73","5 477,23","69 645,42","14 317,84","184 353,14","45 187,28","18 260,97","36 194,12","74 763,20","72 297,22","48 024,33","27 171,72","33 734,76","202 290,60","113 341,98","32 724,99","56 882,16","405,88","6 841,07","124 522,06","42 546,31","7 914,35","11 525,82","24 746,74","28 302,78","14 015,09","337 195,56","44 278,26","1 005 979,08","1 676,77","50 773,24","16 454,66","75 719,70","86 481,15",42.12,45.94,49.85,48.41,"1 001 103,38","559 145,81","23,55","163 207,48",train
3,73,73000000000,73000000000,31.03.2021,73.77,87.81,76.34,32.33,72.69,69.96,38.12,130.84,25.51,56.51,18.93,87.2,44.13,"274,46",166.64,0,319.04,166.56,221.58,"390,08",267.03,235.90,168.87,452.39,55.12,"508,23",7.18,139.74,109.99,11.31,28.90,79.10,"5 387,71","4 578,15","5 325,30","23 452,36","5 516,29","66 617,20","14 603,34","5 681,97","14 970,92","28 312,37","28 872,48","17 826,75","12 461,42","16 097,86","74 719,70","36 675,92","10 394,90","14 839,38",0,"1 750,05","40 283,76","14 760,59","477,36","1 198,71","8 962,07","9 907,19","3 834,77","102 388,62","7 962,17","305 426,75","398,34","16 849,17","6 850,37","27 386,71","25 340,40",43.41,47.01,47.94,46.93,"400 342,50","161 002,06","6 391,44","66 398,72",train
4,65,64000000000,64000000000,31.03.2021,132.15,123.60,109.73,50.71,24.65,20.25,42.23,183.08,56.22,56.38,59.90,190.8,16.20,"391,74",302.58,0,492.83,254.26,334.01,"340,94",320.56,464.63,376.05,745.15,79.74,"654,56",7.65,178.21,148.33,34.19,43.77,101.58,"7 784,41","6 593,31","8 039,06","11 415,69","2 277,09","21 931,80","4 

In [9]:
data = data.fillna(0)
#https://stackoverflow.com/questions/10993612/how-to-remove-xa0-from-string-in-python
#unidecode
for i in data.columns[4:79]:
    data[i] = data[i].apply(lambda x: str(x).replace(',','.'))
    data[i] = data[i].apply(lambda x: str(x).replace(' ',''))
    data[i] = data[i].apply(lambda x: x.replace(u'\xa0', u' '))
    data[i] = data[i].apply(lambda x: str(x).replace(' ',''))
    data[i] = data[i].apply(lambda x: float(x))

In [10]:
data = data.set_index('date')

In [11]:
# Из общего датасета снова выделим тестовую и тренировочную часть
train_data = data.query('sample == "train"').drop(['sample'], axis=1)
test_data = data.query('sample == "test"').drop(['sample'], axis=1)

По частям

Для каждого прогнозируемого показателя (столбца) рассчитывается его среднее значение v_mean j  и значение метрики MAE: v_mae j
* Для каждого показателя вычисляется отношение
  E j  = v_mae j  / v_mean j 
  
* Рассчитывается среднее значение E среди всех столбцов 
  E_mean = 1 / n * sum(E j )
  
* Берется обратная величина и делится на константу 1000
  score = 1 / (1000 * E_mean)

In [12]:
d = []
for i in data.columns[3:77]:
    y = train_data[i]
    n = data.columns.get_loc(i) + 1
    X = train_data.loc[:,data.columns[n:78]]
    #test_data = test_data.loc[:,data.columns[n:78]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)
    
    model = Ridge()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)
    
    print(f'{i}')
    print("Test MAE = %.4f" % mean_absolute_error(y_test, y_pred))
    print("Train MAE = %.4f" % mean_absolute_error(y_train, y_train_pred))
    print("Test median = %.4f" % y_test.median())
    print("Test E = %.4f" % (mean_absolute_error(y_test, y_pred) / y_test.median()))
    d.append((mean_absolute_error(y_test, y_pred) / y_test.median()))
    
    #test_data = test_data.iloc[:,3:77] 
    #x = model.predict(test_data)
    #sample[i] = np.array(model.predict(test_data))

#sample

pasta
Test MAE = 4.5611
Train MAE = 4.5944
Test median = 74.5300
Test E = 0.0612
legumes
Test MAE = 9.4635
Train MAE = 9.5415
Test median = 92.0500
Test E = 0.1028
bread
Test MAE = 2.4252
Train MAE = 2.4615
Test median = 67.3900
Test E = 0.0360
flour
Test MAE = 2.8496
Train MAE = 2.8394
Test median = 33.1100
Test E = 0.0861
rice
Test MAE = 6.1485
Train MAE = 6.1968
Test median = 73.6700
Test E = 0.0835
groats
Test MAE = 6.4516
Train MAE = 6.4652
Test median = 54.0750
Test E = 0.1193
potatoes
Test MAE = 4.7562
Train MAE = 4.7644
Test median = 25.7900
Test E = 0.1844
сucumbers_tomatoes
Test MAE = 20.4985
Train MAE = 20.4370
Test median = 128.9050
Test E = 0.1590
vegetables
Test MAE = 4.7761
Train MAE = 4.7995
Test median = 29.7600
Test E = 0.1605
roots
Test MAE = 5.4173
Train MAE = 5.4506
Test median = 41.8650
Test E = 0.1294
сabbage
Test MAE = 7.6437
Train MAE = 7.6488
Test median = 22.2500
Test E = 0.3435
fruit
Test MAE = 3.7857
Train MAE = 3.8410
Test median = 86.3000
Test E = 0.0439


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in double_scalars


Test E = inf


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in double_scalars


beef
Test MAE = 58.5135
Train MAE = 59.1066
Test median = 352.2700
Test E = 0.1661
chicken
Test MAE = 9.4073
Train MAE = 9.3444
Test median = 155.4300
Test E = 0.0605
pork
Test MAE = 27.8687
Train MAE = 27.6911
Test median = 247.6650
Test E = 0.1125
fish
Test MAE = 80.5095
Train MAE = 80.8682
Test median = 356.7700
Test E = 0.2257
herring
Test MAE = 31.8959
Train MAE = 31.7377
Test median = 222.8150
Test E = 0.1431
curd
Test MAE = 22.8220
Train MAE = 22.9851
Test median = 267.4400
Test E = 0.0853
sour_creame
Test MAE = 20.6126
Train MAE = 20.5655
Test median = 190.9000
Test E = 0.1080
butter
Test MAE = 44.8281
Train MAE = 44.9663
Test median = 510.1450
Test E = 0.0879
milk_kefir
Test MAE = 3.1138
Train MAE = 3.1168
Test median = 54.1600
Test E = 0.0575
cheese
Test MAE = 26.5188
Train MAE = 26.1826
Test median = 484.7950
Test E = 0.0547
egg
Test MAE = 0.5252
Train MAE = 0.5317
Test median = 5.3500
Test E = 0.0982
margarine
Test MAE = 26.2411
Train MAE = 26.4310
Test median = 110.0600
Te

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in double_scalars


Test E = inf


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in double_scalars


beef_value
Test MAE = 665.7543
Train MAE = 664.9125
Test median = 921.9250
Test E = 0.7221
chicken_value
Test MAE = 5529.6560
Train MAE = 5499.6226
Test median = 35748.2550
Test E = 0.1547
pork_value
Test MAE = 3518.7801
Train MAE = 3503.5241
Test median = 8955.6600
Test E = 0.3929
fish_value
Test MAE = 409.2259
Train MAE = 406.8379
Test median = 717.7850
Test E = 0.5701
herring_value
Test MAE = 706.2195
Train MAE = 703.0694
Test median = 1713.5000
Test E = 0.4122
curd_value
Test MAE = 1620.4661
Train MAE = 1643.5916
Test median = 6105.9350
Test E = 0.2654
sour_creame_value
Test MAE = 1932.6764
Train MAE = 1924.5381
Test median = 6710.3800
Test E = 0.2880
butter_value
Test MAE = 911.0540
Train MAE = 914.4575
Test median = 3162.6100
Test E = 0.2881
milk_kefir_value
Test MAE = 14327.1043
Train MAE = 14303.3800
Test median = 78796.8400
Test E = 0.1818
cheese_value
Test MAE = 1700.7357
Train MAE = 1694.1671
Test median = 6611.1300
Test E = 0.2573
egg_value
Test MAE = 82537.8065
Train MAE =

In [13]:
d = pd.DataFrame(d)
d[0]=d[0].replace(np.inf,0)
E_mean = 1 / 32 * d[0].sum()
score = 1 / (1000 * E_mean)
print(E_mean, score)

0.7264635428121872 0.001376531568437606


In [14]:
len(test_data)

7735

In [15]:
test_data.shape

(7735, 78)

In [16]:
train_data.shape

(69785, 78)

In [17]:
sample_submission.shape

(7735, 79)

In [14]:
y = train_data.loc[:,data.columns[3:35]]
X = train_data.loc[:,data.columns[35:78]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)
    
model = Ridge()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)
    
print("Test MAE = %.4f" % mean_absolute_error(y_test, y_pred))
print("Train MAE = %.4f" % mean_absolute_error(y_train, y_train_pred))

test_data = test_data.loc[:,test_data.columns[35:78]]
predict_submission = model.predict(test_data)
len(predict_submission)

Test MAE = 20.3070
Train MAE = 20.3342


7735